In [8]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import tensorflow as tf
import os 
from keras.models import Sequential 
from tensorflow import keras
from keras.layers import Dense, Conv2D, Flatten, MaxPool2D
from PIL import Image
from sklearn.model_selection import train_test_split
import random

tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU')]

In [9]:
def hk_label_conversion(label) :
  # Joseph Lin 2/18/2023
  # This function converts an integer label of the image to the string equivalent classes 
  # Function input: 
  # label: an integer number specifying the classes of the image 
  # Function output: 
  # returns a string equivalent of the label of the image
  if label == 0 : 
    return "10 dollars"
  elif label == 1: 
    return "20 dollars"
  elif label == 2: 
    return "50 dollars" 
  elif label == 3: 
    return "100 dollars" 
  elif label == 4: 
    return "500 dollars" 
  elif label == 5: 
    return "1000 dollars" 

In [10]:
def load_hk_data(directory, getIndividual=None): 
  # Joseph Lin 2/18/2023
  # This function loads the thai dataset from the image folder 
  # do the transformation to the image (resizing and normalizing) 
  # then split the image into training and testing dataset
  # Function inputs: 
  # directory: the path of the folder that contains all the thai images
  # getIndividual: a boolean specifiying which kind of data should be returned

  # image and label for each classes of bank notes
  img_10 = [] 
  lbl_10 = [] 
  img_20 = [] 
  lbl_20 = [] 
  img_50 = [] 
  lbl_50 = [] 
  img_100 = [] 
  lbl_100 = [] 
  img_500 = [] 
  lbl_500 = []
  img_1000 = []
  lbl_1000 = [] 
  # initialize the training size 
  training_size = 0.8

  for image_class in os.listdir(directory) : 
    for image_name in os.listdir(os.path.join(directory, image_class)):
      # open the image at the image file path
      image = Image.open(os.path.join(directory, image_class, image_name))
      # resize the image to 32x32
      image.thumbnail((300,300)) 
      # normalize the image
      image = np.array(image) / 255 
      if image_class == "10 dollars": # if the image belongs to 10 hk dollars
        lbel = 0
        img_10.append(image) 
        lbl_10.append(lbel) 
      elif image_class == "20 dollars": # if the image belongs to 20 hk dollars 
        lbel = 1
        img_20.append(image) 
        lbl_20.append(lbel) 
      elif image_class == "50 dollars": # if the image belongs to 50 hk dollars 
        lbel = 2
        img_50.append(image)
        lbl_50.append(lbel)       
      elif image_class == "100 dollars": # if the image belongs to 100 hk dollars 
        lbel = 3
        img_100.append(image) 
        lbl_100.append(lbel) 
      elif image_class == "500 dollars": # if the image belongs to 500 hk dollars 
        lbel = 4
        img_500.append(image) 
        lbl_500.append(lbel) 
      elif image_class == "1000 dollars": # if the image belongs to 1000 hk dollars 
        lbel = 5
        img_1000.append(image) 
        lbl_1000.append(lbel) 

  # equally split the dataset into train and test for each classes of images
  hk10_img_train, hk10_img_test, hk10_label_train, hk10_label_test = train_test_split(img_10, lbl_10, train_size=training_size) 
  hk20_img_train, hk20_img_test, hk20_label_train, hk20_label_test = train_test_split(img_20, lbl_20, train_size=training_size) 
  hk50_img_train, hk50_img_test, hk50_label_train, hk50_label_test = train_test_split(img_50, lbl_50, train_size=training_size) 
  hk100_img_train, hk100_img_test, hk100_label_train, hk100_label_test = train_test_split(img_100, lbl_100, train_size=training_size)
  hk500_img_train, hk500_img_test, hk500_label_train, hk500_label_test = train_test_split(img_500, lbl_500, train_size=training_size) 
  hk1000_img_train, hk1000_img_test, hk1000_label_train, hk1000_label_test = train_test_split(img_1000, lbl_1000, train_size=training_size) 
 
  
  # combine the data together into train and test set
  x_train = np.vstack([hk10_img_train, hk20_img_train, hk50_img_train, hk100_img_train, hk500_img_train, hk1000_img_train])
  y_train = np.hstack([hk10_label_train, hk20_label_train, hk50_label_train, hk100_label_train, hk500_label_train, hk1000_label_train])
  x_test = np.vstack([hk10_img_test, hk20_img_test, hk50_img_test, hk100_img_test, hk500_img_test, hk1000_img_test])
  y_test = np.hstack([hk10_label_test, hk20_label_test, hk50_label_test, hk100_label_test, hk500_label_test, hk1000_label_test])

  return np.array(x_train), np.array(y_train), np.array(x_test), np.array(y_test) 


In [11]:
# Leng Lohanakakul 11/7/2022
# Main function for loading the combined and individual dataset

data_dir = '/data/lin1223/Dataset'

# load the dataset and split it into train and test set 
x_thai_train, y_thai_train, x_thai_test, y_thai_test = load_hk_data(data_dir, False)

# number of training and testing dataset 
print(f"number of Thai training images = {y_thai_train.shape[0]}")
print(f"number of Thai testing images = {y_thai_test.shape[0]}")

number of Thai training images = 1176
number of Thai testing images = 294


In [12]:
# Leng Lohanakakul 11/7/2022
# Define a convolutional neural network that takes an input shape of 128x128x3
cnn = Sequential() 
cnn.add(Conv2D(32, kernel_size=3, activation='relu', input_shape=(225,300,3), name='conv1'))
cnn.add(Conv2D(64, kernel_size=3, activation='relu', name='conv2'))
cnn.add(MaxPool2D(pool_size=(4,4), name='pool2'))
cnn.add(Flatten(name='flat'))
cnn.add(Dense(7, activation='softmax', name='output'))

cnn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1 (Conv2D)              (None, 223, 298, 32)      896       
                                                                 
 conv2 (Conv2D)              (None, 221, 296, 64)      18496     
                                                                 
 pool2 (MaxPooling2D)        (None, 55, 74, 64)        0         
                                                                 
 flat (Flatten)              (None, 260480)            0         
                                                                 
 output (Dense)              (None, 7)                 1823367   
                                                                 
Total params: 1,842,759
Trainable params: 1,842,759
Non-trainable params: 0
_________________________________________________________________


In [14]:
# Leng Lohanakakul 11/7/2022
# Main function for compiling the model and defining the hyperparameters 

# define the optimizer and loss function used for the neural network
cnn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# train the model with training dataset over n_epochs 
print(y_thai_train.shape)
print(y_thai_test.shape)

x_thai_train = np.asarray(x_thai_train).astype(np.float32)
y_thai_train = np.asarray(y_thai_train).astype(np.float32)

n_epochs = 15
training = cnn.fit(x_thai_train, y_thai_train, epochs=n_epochs)

#save model to a file 
# cnn.save("/content/drive/My Drive/VIP/model3_uae.h5")

(1176,)
(294,)


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).